Genome Deep Learning

In [1]:
import os

import tensorflow as tf
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior() 
import BRCA_inference
import BRCA_batch
import BRCA_data

In [2]:
# 配置神经网络参数
# Set Neuronetwork Parameters
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARAZTION_RATE = 0.0001
TRAINING_STEPS = 600
MOVING_AVERAGE_DECAY = 0.99

In [3]:
# 数据读取路径
#Data_Read_PATH = "/home/sunysh/12Cancer/BRCA/10000-1-vcf/part1/train.Matrix"

Data_Read_PATH = "C:\\Users\\Eric\\Desktop\\Final_project_code_data\\10000-1-vcf\\10000-1-vcf\\part1\\train.Matrix"

# 模型保存的路径和文件名
#MODEL_SAVE_PATH = "/home/sunysh/12Cancer/BRCA/10000-1-vcf/part1/Model"

MODEL_SAVE_PATH = "C:\\Users\\Eric\\Desktop\\Final_project_code_data\\10000-1-vcf\\10000-1-vcf\\part1\\Model"
MODEL_NAME = "model.ckpt"

In [4]:

def train():
    # 训练数据读取
    L = BRCA_data.readCase(Data_Read_PATH)

    #定义输入层
    x = tf.placeholder(tf.float32, [None, BRCA_inference.INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, BRCA_inference.OUTPUT_NODE], name='y_input')

    # 返回regularizer函数，L2正则化项的值
    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    # 使用BRCA_inference.py中定义的前向传播过程
    y = BRCA_inference.inference(x, regularizer)
    # 定义step为0
    global_step = tf.Variable(0, trainable=False)

    # 滑动平均,由衰减率和步数确定
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    # 可训练参数的集合
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

    # 交叉熵损失函数
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    # 交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    # 总损失
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))

    # 学习率(衰减)
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, TRAINING_STEPS, LEARNING_RATE_DECAY)
    # 定义了反向传播的优化方法，之后通过sess.run(train_step)就可以对所有GraphKeys.TRAINABLE_VARIABLES集合中的变量进行优化，似的当前batch下损失函数更小
    # 实现梯度下降算法的优化器
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    # 更新参数
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    saver = tf.train.Saver()

    #TensorBoard
    writer = tf.summary.FileWriter("C:\\Users\\Eric\\Desktop\\Final_project_code_data\\10000-1-vcf\\10000-1-vcf\\part1\\Log", tf.get_default_graph())
    writer.close()

    # 初始会话，并开始训练过程
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        for i in range(TRAINING_STEPS):
            L = BRCA_data.readCase(Data_Read_PATH)
            xs, ys = BRCA_batch.all(L)
            op, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
            print(global_step.eval(), 'loss:', loss_value)
            
        saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)

        #saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)




In [5]:

train()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
1 loss: 1.6909235
2 loss: 1.7451932
3 loss: 1.5935268
4 loss: 1.4768261
5 loss: 1.4373531
6 loss: 1.4020199
7 loss: 1.3684076
8 loss: 1.336546
9 loss: 1.3070184
10 loss: 1.2800237
11 loss: 1.255716
12 loss: 1.2341719
13 loss: 1.2155734
14 loss: 1.1997017
15 loss: 1.1872715
16 loss: 1.1791923
17 loss: 1.182256
18 loss: 1.2062185
19 loss: 1.3378562
20 loss: 1.6758127
21 loss: 2.3979387
22 loss: 1.293473
23 loss: 1.2553895
24 loss: 1.2307898
25 loss: 1.2119154
26 loss: 1.1965839
2

344 loss: 0.98809284
345 loss: 0.9879342
346 loss: 0.9877755
347 loss: 0.98761696
348 loss: 0.98745835
349 loss: 0.98729974
350 loss: 0.98714125
351 loss: 0.98698276
352 loss: 0.9868244
353 loss: 0.98666596
354 loss: 0.98650765
355 loss: 0.98634946
356 loss: 0.9861912
357 loss: 0.98603296
358 loss: 0.9858747
359 loss: 0.98571664
360 loss: 0.9855586
361 loss: 0.98540056
362 loss: 0.9852426
363 loss: 0.9850847
364 loss: 0.98492676
365 loss: 0.98476887
366 loss: 0.98461103
367 loss: 0.98445326
368 loss: 0.9842955
369 loss: 0.9841377
370 loss: 0.9839802
371 loss: 0.9838226
372 loss: 0.98366493
373 loss: 0.98350734
374 loss: 0.98334986
375 loss: 0.98319244
376 loss: 0.98303497
377 loss: 0.98287755
378 loss: 0.98272014
379 loss: 0.98256296
380 loss: 0.9824056
381 loss: 0.9822484
382 loss: 0.9820911
383 loss: 0.98193413
384 loss: 0.98177695
385 loss: 0.98161983
386 loss: 0.9814629
387 loss: 0.98130584
388 loss: 0.98114884
389 loss: 0.98099196
390 loss: 0.98083514
391 loss: 0.98067826
392 loss